## 1st variant
### Dynamic Bayesian Network Structure Learning with hybrid network = (classic) encoder + (quantum) circuit.
This is a toy example with dummy generated training data of 4 variables, that means 8 vertices (4 for t and 4 for t+1).    
n_qubits=6  
where first three digits correspond to vertice where the edge begins and three las digits correspond to vertice qhere edge ends i.e. 010101 corresponds to: vertice_2 ---> vertice_5  
It was tested with two sets of test data at the end of the notebook. The last test is the mean of 7 experiments of the same network, where each experiment is build with data for t and t+1  


In [1]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import torchvision
from torch import nn
import numpy as np
import pennylane as qml
import random

In [24]:
path = "tiny_dummy_qhack.csv"

In [25]:
ds = pd.read_csv(path)
ds

,vertices,experiment,t0,t1
0,v0,1,0.50,0.10
1,v1,1,0.03,0.45
2,v2,1,0.50,0.04
3,v3,1,0.60,0.80
4,v0,2,0.10,0.07
5,v1,2,0.45,0.10
6,v2,2,0.04,0.04
7,v3,2,0.80,0.50
8,v0,3,0.07,0.05
9,v1,3,0.10,0.06


In [26]:
def get_edges(n=4):
    num_edges = random.randint(n, n+3)
    e1 = [(random.randint(0, n-1),random.randint(0, (n*2)-1)) for f in range(num_edges//2)]
    e2 = [(random.randint(0, (n*2)-1),random.randint(n, (n*2)-1)) for f in range(num_edges//2)]
    return e1 + e2

def get_t0(edges, n=4):
    t0 = np.zeros(n) + 0.01
    edges0 = [edge for i in range(n) for edge in edges if edge[0] == i and edge[1] < n]
    if len(edges0) > 0:
        t0[edges0[0][0]] = random.random()
        for edge in edges0:
            t0[edge[1]] += t0[edge[0]]
    return t0
        
def get_t1(edges, t0, n=4):
    t1 = np.zeros(n) + 0.01
    edges1 = [edge for edge in edges if edge[1] >= n]
    for edge in edges1:
        if edge[0] < n:
            t1[edge[1]-n] += t0[edge[0]]
        else:
            t1[edge[1]-n] += t1[edge[0]-n]
    return t1

In [27]:
# generate training dataset
exper = 300
nnodes = 4
n_qubits = 6
arr_list = []
edges_list = []
for f in range(exper):
    edges = get_edges(n = nnodes)
    t0 = get_t0(edges, n = nnodes)
    t1 = get_t1(edges, t0, n = nnodes)
    arr_list.append(np.stack([t0,t1]))
    edges_list.append(edges)
arr = np.concatenate(arr_list, axis=1)

In [28]:
dsa = pd.DataFrame({'t01':arr_list})
dsa

,t01
0,"[[0.47419259402578307, 0.01, 0.474192594025783..."
1,"[[0.01, 0.01, 0.01, 0.01], [0.02, 0.01, 0.04, ..."
2,"[[0.7650931994812707, 0.7550931994812707, 0.01..."
3,"[[0.01, 0.18999161913056528, 0.01, 0.209991619..."
4,"[[0.8235732991036843, 0.8335732991036843, 0.01..."
...,...
295,"[[0.01, 0.01, 0.01, 0.01], [0.03, 0.01, 0.03, ..."
296,"[[0.01, 0.01, 0.04277581285149856, 0.095551625..."
297,"[[0.20534981110932282, 0.1953498111093228, 0.0..."
298,"[[0.01, 0.01, 0.01, 0.01], [0.01, 0.03, 0.01, ..."


In [29]:
#int("110100010",2) = 418
edges_bin_list = [[np.binary_repr(ed[0], width=n_qubits//2) + np.binary_repr(ed[1], width=n_qubits//2)  for ed in edges] for edges in edges_list]
ya_list = [[int(edge,2) for edge in edges] for edges in edges_bin_list]

In [30]:
dsa['y'] = ya_list

In [31]:
dsa

,t01,y
0,"[[0.47419259402578307, 0.01, 0.474192594025783...","[24, 26, 29, 31, 39, 45]"
1,"[[0.01, 0.01, 0.01, 0.01], [0.02, 0.01, 0.04, ...","[7, 12, 22, 54]"
2,"[[0.7650931994812707, 0.7550931994812707, 0.01...","[12, 8, 60, 29]"
3,"[[0.01, 0.18999161913056528, 0.01, 0.209991619...","[19, 11, 6, 20, 46, 12]"
4,"[[0.8235732991036843, 0.8335732991036843, 0.01...","[30, 1, 60, 62]"
...,...,...
295,"[[0.01, 0.01, 0.01, 0.01], [0.03, 0.01, 0.03, ...","[20, 15, 6, 4, 55, 30]"
296,"[[0.01, 0.01, 0.04277581285149856, 0.095551625...","[19, 19, 28, 37]"
297,"[[0.20534981110932282, 0.1953498111093228, 0.0...","[30, 31, 8, 55, 28, 29]"
298,"[[0.01, 0.01, 0.01, 0.01], [0.01, 0.03, 0.01, ...","[7, 21, 13, 55]"


In [32]:
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

In [33]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

In [34]:
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [35]:
input_size = nnodes * 2
hidden_size = input_size - 2
code_size = n_qubits
encoder_hidden_layer = nn.Linear(
            in_features=input_size, out_features=hidden_size
        )
encoder_output_layer = nn.Linear(
            in_features=hidden_size, out_features=code_size
        )

In [13]:
layers = [encoder_hidden_layer, encoder_output_layer, qlayer]
model = torch.nn.Sequential(*layers)

In [37]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.2)
#criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [38]:
class CustomDataset(Dataset):
    def __init__(self, ds, n, q, transform=None):
        self.ds_full = ds
        self.n = n
        self.q = q
        self.x_csv = self.ds_full[["t01"]]
        self.y_csv = self.ds_full[["y"]]
        self.transform = transform

    def __len__(self):
        return len(self.x_csv)

    def __getitem__(self, idx):
        x = np.array(self.x_csv.iloc[idx].tolist()[0])
        y = np.zeros(2**self.q)
        for i in self.y_csv.iloc[idx].tolist()[0]:
            #011000 24
            y[i] = 1
        if self.transform:
            x = self.transform(x)
            y = self.transform(y)
        return x, y

In [39]:
batch_size = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

train_dataset = CustomDataset(dsa, nnodes, n_qubits, transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False
)

In [40]:
epochs=30
for epoch in range(epochs):
    loss = 0
    for batch_features, y_batch in train_loader:
        batch_features = batch_features.view(-1, input_size)
        
        optimizer.zero_grad()
        
        outputs = model(batch_features)
        
        train_loss = criterion(outputs, y_batch)
        
        train_loss.backward()
        
        optimizer.step()
        
        loss += train_loss.item()
    
    loss = loss / len(train_loader)
    
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/30, loss = 0.073198
epoch : 2/30, loss = 0.072905
epoch : 3/30, loss = 0.072750
epoch : 4/30, loss = 0.072641


KeyboardInterrupt: 

In [41]:
batch_features

tensor([[0.3756, 0.0100, 0.3656, 0.0200, 0.3856, 0.0100, 0.0200, 0.3956]])

In [42]:
batch_features

tensor([[0.3756, 0.0100, 0.3656, 0.0200, 0.3856, 0.0100, 0.0200, 0.3956]])

In [43]:
outputs = model(batch_features)

In [44]:
outputs

tensor([[0.0007, 0.0077, 0.0177, 0.0085, 0.0438, 0.0296, 0.0092, 0.0360, 0.0071,
         0.0093, 0.0037, 0.0042, 0.0249, 0.0526, 0.0161, 0.0430, 0.0122, 0.0211,
         0.0011, 0.0152, 0.0077, 0.0436, 0.0259, 0.0216, 0.0072, 0.0304, 0.0112,
         0.0021, 0.0342, 0.0463, 0.0429, 0.0146, 0.0025, 0.0123, 0.0015, 0.0018,
         0.0110, 0.0081, 0.0157, 0.0225, 0.0077, 0.0032, 0.0117, 0.0003, 0.0133,
         0.0083, 0.0124, 0.0372, 0.0113, 0.0049, 0.0123, 0.0057, 0.0106, 0.0164,
         0.0098, 0.0024, 0.0203, 0.0131, 0.0197, 0.0004, 0.0058, 0.0009, 0.0054,
         0.0399]], grad_fn=<StackBackward0>)

In [45]:
np.flip(np.argsort(outputs.detach().numpy()))

array([[13, 29,  4, 21, 15, 30, 63, 47,  7, 28, 25,  5, 22, 12, 39, 23,
        17, 56, 58,  2, 53, 14, 38, 19, 31, 44, 57, 46, 50, 33, 16, 42,
        48, 26, 36, 52, 54,  9,  6,  3, 45, 37,  1, 40, 20, 24,  8, 60,
        51, 62, 49, 11, 10, 41, 32, 55, 27, 35, 34, 18, 61,  0, 59, 43]])

In [46]:
np.nonzero(y_batch)

tensor([[ 0,  2],
        [ 0,  4],
        [ 0, 24],
        [ 0, 27],
        [ 0, 39],
        [ 0, 62]])

## testing

In [53]:
path = "tiny_dummy_qhack_test.csv"

In [54]:
dstemp = pd.read_csv(path)
dstemp

,vertices,experiment,t0,t1
0,v0,1,0.060,0.060
1,v1,1,0.070,0.070
2,v2,1,0.500,0.065
3,v3,1,0.060,0.450
4,v0,2,0.060,0.050
5,v1,2,0.070,0.060
6,v2,2,0.065,0.060
7,v3,2,0.450,0.060
8,v0,3,0.050,0.050
9,v1,3,0.060,0.060


In [62]:
t01_list = [dstemp[['t0','t1']].iloc[f*4:(f+1)*4].values.T for f in range(4)]
y_list = [[8,25,7,6] for i in range(4)]
dst = pd.DataFrame({'t01':t01_list,'y':y_list})
dst

,t01,y
0,"[[0.06, 0.07, 0.5, 0.06], [0.06, 0.07, 0.065, ...","[8, 25, 7, 6]"
1,"[[0.06, 0.07, 0.065, 0.45], [0.05, 0.06, 0.06,...","[8, 25, 7, 6]"
2,"[[0.05, 0.06, 0.06, 0.06], [0.05, 0.06, 0.05, ...","[8, 25, 7, 6]"
3,"[[0.05, 0.06, 0.05, 0.05], [0.05, 0.06, 0.05, ...","[8, 25, 7, 6]"


In [63]:
n = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

test_dataset = CustomDataset(dst, nnodes, n_qubits, transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True
)


In [66]:
experiments = []
outputs_list = []
for batch_features, _ in test_loader:
    batch_features = batch_features.view(-1, input_size)
    batch_features
    outputs = model(batch_features)
    outputs_list.append(outputs)
    experiments.append(np.flip(np.argsort(outputs.detach().numpy())))
experiments

[array([[13, 15, 29, 21,  4, 30, 63, 47,  7, 28, 25,  5, 22, 12, 39, 23,
         58, 17, 56,  2, 14, 38, 19, 53, 31, 44, 57, 26, 50, 42, 33, 46,
         16, 48, 52, 36,  6,  3, 54,  9, 37, 45, 20,  1, 24, 60, 40,  8,
         49, 10, 62, 51, 41, 11, 55, 27, 32, 35, 34, 18, 61,  0, 43, 59]]),
 array([[13, 29, 21,  4, 30, 15, 63, 25, 47, 28,  7,  5, 17, 22, 12, 39,
         56, 53, 23, 38,  2, 19, 58, 14, 31, 16, 54, 33, 46, 48, 36, 57,
          9,  8, 52, 44, 40, 50, 45,  6, 37, 24, 20,  1, 42, 51,  3, 11,
         60, 26, 62, 49, 41, 32, 27, 35, 10, 34, 55,  0, 61, 59, 18, 43]]),
 array([[13, 29,  4, 21, 30, 63, 15,  7,  5, 47, 28, 25, 17, 12, 39, 22,
         56, 23, 53, 58, 38, 14, 19,  2, 31, 54, 46, 16,  8, 57, 40, 36,
         48, 44, 33, 45, 52, 50,  9,  6, 24, 37,  1, 42, 20,  3, 51, 11,
         62, 26, 60, 32, 49, 41, 35,  0, 55, 61, 27, 10, 18, 34, 59, 43]]),
 array([[13, 29,  4, 21, 30, 63, 15,  7,  5, 47, 28, 25, 17, 12, 39, 56,
         22, 23, 53, 58, 38, 14, 19,  2, 3

In [85]:
results = np.flip(np.argsort(sum(outputs_list).detach().numpy()))

In [79]:
np.max(sum(outputs_list).detach().numpy()),np.min(sum(outputs_list).detach().numpy())

(0.21034093, 0.0015496378)

In [84]:
sum(outputs_list)[0][25]

tensor(0.1255, grad_fn=<SelectBackward0>)

In [91]:
results_bin = [np.binary_repr(f, width=n_qubits) for f in results.tolist()[0]]

In [97]:
results_bin[:12]

['001101',
 '011101',
 '000100',
 '010101',
 '011110',
 '001111',
 '111111',
 '101111',
 '000111',
 '011100',
 '000101',
 '011001']

In [96]:
results.tolist()[0][:12]

[13, 29, 4, 21, 30, 15, 63, 47, 7, 28, 5, 25]

## testing2

In [98]:
ds

,vertices,experiment,t0,t1
0,v0,1,0.50,0.10
1,v1,1,0.03,0.45
2,v2,1,0.50,0.04
3,v3,1,0.60,0.80
4,v0,2,0.10,0.07
5,v1,2,0.45,0.10
6,v2,2,0.04,0.04
7,v3,2,0.80,0.50
8,v0,3,0.07,0.05
9,v1,3,0.10,0.06


In [100]:
t01_list = [ds[['t0','t1']].iloc[f*4:(f+1)*4].values.T for f in range(7)]
y_list = [[24,19,5,15,23] for i in range(7)]
dst2 = pd.DataFrame({'t01':t01_list,'y':y_list})
dst2

,t01,y
0,"[[0.5, 0.03, 0.5, 0.6], [0.1, 0.45, 0.04, 0.8]]","[24, 19, 5, 15, 23]"
1,"[[0.1, 0.45, 0.04, 0.8], [0.07, 0.1, 0.04, 0.5]]","[24, 19, 5, 15, 23]"
2,"[[0.07, 0.1, 0.04, 0.5], [0.05, 0.06, 0.03, 0....","[24, 19, 5, 15, 23]"
3,"[[0.05, 0.06, 0.03, 0.15], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"
4,"[[0.04, 0.04, 0.03, 0.1], [0.04, 0.04, 0.03, 0...","[24, 19, 5, 15, 23]"
5,"[[0.04, 0.04, 0.03, 0.07], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"
6,"[[0.04, 0.04, 0.03, 0.07], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"


In [103]:
n = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

test_dataset = CustomDataset(dst2, nnodes, n_qubits, transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True
)


In [104]:
experiments = []
outputs_list = []
for batch_features, _ in test_loader:
    batch_features = batch_features.view(-1, input_size)
    batch_features
    outputs = model(batch_features)
    outputs_list.append(outputs)
    experiments.append(np.flip(np.argsort(outputs.detach().numpy())))
experiments

[array([[13, 15, 29, 21,  4, 63, 30, 47, 28,  7, 25, 22,  5, 12, 39,  2,
         58, 23, 31, 56, 14, 33, 16, 19, 38, 17, 57, 44, 26, 42, 46, 53,
         50, 52,  9, 48, 36, 54,  6, 20,  3, 10, 45, 60, 37, 49,  1, 24,
         51, 40,  8, 62, 27, 34, 11, 55, 41, 18, 35, 61, 43, 32, 59,  0]]),
 array([[13, 21, 29,  4, 25, 15, 63, 30,  5, 47, 28, 22,  7, 39, 12, 17,
         19,  2, 56, 38, 31, 53, 16, 33, 54,  9, 14, 52, 36, 48, 46, 23,
          8, 57, 58, 50,  6, 45, 24, 40, 37,  1, 20, 44, 11, 60, 51, 42,
          3, 26, 62, 49, 27, 10, 34, 41, 59, 32, 35, 55, 18, 61,  0, 43]]),
 array([[13, 29, 21,  4, 30, 15, 63, 25, 47, 28,  7,  5, 17, 22, 12, 39,
         56, 53,  2, 19, 38, 23, 31, 14, 58, 16, 33, 54, 46,  9, 48, 36,
         57, 52,  8, 44, 40, 50, 45,  6, 24, 37, 20,  1, 42, 11, 51,  3,
         60, 26, 62, 49, 41, 32, 27, 10, 34, 35, 55, 59,  0, 61, 18, 43]]),
 array([[13, 29,  4, 21, 30, 63, 15,  7, 47,  5, 28, 25, 17, 12, 39, 22,
         56, 53, 23, 58, 38, 19,  2, 14, 3

In [105]:
results = np.flip(np.argsort(sum(outputs_list).detach().numpy()))

In [106]:
np.max(sum(outputs_list).detach().numpy()),np.min(sum(outputs_list).detach().numpy())

(0.36945578, 0.002905326)

In [107]:
results_bin = [np.binary_repr(f, width=n_qubits) for f in results.tolist()[0]]

In [116]:
sum(outputs_list)[0][5]

tensor(0.2230, grad_fn=<SelectBackward0>)

In [114]:
results.tolist()[0][:12]

[13, 29, 4, 21, 30, 15, 63, 25, 47, 7, 28, 5]

In [ ]:
[24,19,5,15,23]

In [115]:
results_bin[:12]

['001101',
 '011101',
 '000100',
 '010101',
 '011110',
 '001111',
 '111111',
 '011001',
 '101111',
 '000111',
 '011100',
 '000101']